In [ ]:
import os
#os.environ["MKL_NUM_THREADS"] = "50"
#os.environ["NUMEXPR_NUM_THREADS"] = "50"
#os.environ["OMP_NUM_THREADS"] = "50"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline

import loompy
import scanpy as sc
import numpy as np
import pandas as pd
import re
import scipy.sparse
import bbknn
import scanpy.external as sce
from gprofiler import GProfiler
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
sc.set_figure_params(dpi = 150, dpi_save = 300, format = 'png')
sc._settings.ScanpyConfig(verbosity=0)

#############################################################################
# COMPILE COMMAND LINE ######################################################
#jupyter nbconvert --ExecutePreprocessor.timeout=80000 --execute --to html
#############################################################################

In [ ]:
### SET CONSTANSTS

In [ ]:
# Do the loom files need to be merged prior to anything?
Need_to_combine = False
if Need_to_combine:
    Experiment_names = [] #LIST HERE YOUR EXPERIMENT NAMES e.g. "d0es","d1d",etc.. as they appear in the loom file name
    FILE_DIR = ""# LOCATION OF YOUR LOOM FILES

OUTPUT_FILE_DIR = "/scratch/gaurav/loom-files" # THE DIRECTORY YOUR COMBINED LOOM FILE IS IN
loompy_output = "d0-1-2-14.loom" # YOUR COMBINED LOOM FILE NAME

# PROVIDE CELL CYCLE GENE LISTS FOR PHASE SCORING
S_genes_file = "~/s_genes.csv"
G2M_genes_file = "~/g2m_genes.csv"
S_phase_genes = pd.read_csv(S_genes_file)["x"].to_list()
G2M_phase_genes = pd.read_csv(S_genes_file)["x"].to_list()

# HERE YOU CAN CHOOSE A FEW HYPERPARAMETERS FOR YOUR ANALYSIS
BBKNN_neighbors = 5
ClusteringResolution = 0.5

gp = GProfiler(return_dataframe=True)
ExportEmbeddingClusteringInfo = True # WHETHER TO EXPORT THE EMBEDDING AND CLUSTERING INFO YOU WILL COMPUTE FOR EXTERNAL ANALYSES (e.g. Monocle etc..)

In [ ]:
#############DEFINE FUNCTIONS########
def Add_obs(loaded_obj):
    mito_genes = loaded_obj.var_names.str.startswith(('MT-', 'MTRNR'))
    ribo_genes = loaded_obj.var_names.str.startswith(('RPL','RPS')) #, 'RP[0-5][0-9]'))
    loaded_obj.obs['percent_mito'] = np.sum(loaded_obj[:, mito_genes].X, 
                                            axis=1).A1 / np.sum(loaded_obj.X, axis=1).A1
    loaded_obj.obs['percent_ribo'] = np.sum(loaded_obj[:, ribo_genes].X, 
                                            axis=1).A1 / np.sum(loaded_obj.X, axis=1).A1
    loaded_obj.obs['n_counts'] = loaded_obj.X.sum(axis=1).A1
    loaded_obj.obs["SampleID"] = ["d2d" if ID[0] == "48hrsd" else 
                                  "d14d-1" if ID[0] == "d14dc" else 
                                  "d14d-2" if ID[0] == "d14-d-d" else 
                                  "d14d-3" if ID[0] == "d14-d-e" else 
                                  ID[0] for ID in loaded_obj.obs_names.str.split("-5")]
    
    return(loaded_obj)

def Get_genes_to_keep(loaded_obj):
    detected_genes = loaded_obj[:,sc.pp.filter_genes(loaded_obj, min_cells=3, 
                                                     inplace=False)[0]].var_names.tolist()
    MitoRiboGenes = (list(filter(lambda x: re.search(r'^MT-', x, re.IGNORECASE), detected_genes)) + 
                     list(filter(lambda x: re.search(r'^MTRNR', x, re.IGNORECASE), detected_genes)) + 
                     list(filter(lambda x: re.search(r'^RPL', x, re.IGNORECASE), detected_genes)) +
                     # "^RP[0-5][0-9]" catches a lot of non ribosomal proteins like LINCs, Pseudogenes and even prot coding genes
                     #list(filter(lambda x: re.search(r'^RP[0-5][0-9]', x, re.IGNORECASE), detected_genes)) +
                     list(filter(lambda x: re.search(r'^RPS', x, re.IGNORECASE), detected_genes)))
    return(list(set(detected_genes) - set(MitoRiboGenes)))

def plot_pca_loadings_customized(adata, n_genes, n_comp):
    ind = np.argsort(adata.varm['PCs'][:,0:n_comp], axis=0)[-n_genes:][::-1] # get indeces of top 20 genes in top 5 PCs
    plotting_df = pd.DataFrame(columns=['Gene','PC_importance', 'Component'])
    for n in range(n_comp):
        component_number = n + 1
        plotting_df = pd.concat([plotting_df,
                    pd.DataFrame(np.vstack((adata.var.iloc[ind[:,n]].index, 
                                            adata.varm['PCs'][ind[:,n],n], 
                                            ["PC {}".format(component_number)]*n_genes)).T, 
                                 columns=['Gene','PC_importance', 'Component'])])

    g = sns.FacetGrid(plotting_df, col="Component", sharey=False, height=5)
    g = g.map(plt.scatter, "PC_importance", "Gene", edgecolor="w")

def PCA_plotting_wrapper(loaded_obj):
    sc.pl.pca_variance_ratio(loaded_obj, log=True)
    plot_pca_loadings_customized(loaded_obj, 20, 5)
    sc.pl.pca(loaded_obj, color = ['SampleID'], legend_loc = 'on data', use_raw = False, 
              components = ['1,2', '2,3', '3,4'], ncols = 3, wspace = 0, 
              title = ["PC1 vs PC2", "PC2 vs PC3", "PC3 vs PC4"])
    
def PreProcessing(obj, QCplots=False, Verbosity="high"):
    obj = Add_obs(obj)
  
    if QCplots:
        sc.pl.violin(obj, ['percent_mito', 'percent_ribo', 'n_counts'], jitter=0.4, multi_panel=True)

    # first round of gene filtering
    Genes_to_keep = Get_genes_to_keep(obj)
    obj = obj[:,Genes_to_keep]
    if Verbosity=="high":
        print("First gene filtering: "+str(obj.X.shape))

    # Cells filtering
    sc.pp.filter_cells(obj, min_genes=200)
    median_genes_per_cell = np.median(obj.X.getnnz(axis=1)) # recomputing this per each dataset will introduce inconsistent thrholds
    sc.pp.filter_cells(obj, max_genes=2.5*median_genes_per_cell)
    if Verbosity=="high":
        print("Cell filtering: "+str(obj.X.shape))

    # Second round of gene filtering: some genes got to all-zeros after cell filtering
    Genes_to_keep_2 = Get_genes_to_keep(obj)
    obj = obj[:,Genes_to_keep_2]
    if Verbosity=="high":
        print("Second gene filtering: "+str(obj.X.shape))

    obj = Add_obs(obj)
    if QCplots:
        sc.pl.violin(obj, ['percent_mito', 'percent_ribo', 
                           'n_counts', 'n_genes'], jitter=0.4, multi_panel=True)

    if Verbosity=="high":
        print("Raw!")
        print(obj.X[:10,:10])

    sc.pp.normalize_total(obj, layers="all")
    if Verbosity=="high":
        print("Normalized")
        print(obj.X[:10,:10])

    sc.pp.log1p(obj)
    if Verbosity=="high":
        print("Log transformed")
        print(obj.X[:10,:10])

    sc.pp.highly_variable_genes(obj, subset=True) # needs log transformed data
    if QCplots:
        sc.pl.highly_variable_genes(obj)
    if Verbosity=="high":
        print("HVG subset: "+str(obj.shape))
    
    return(obj)

def CellCycleRegression(obj, CCregr):
    
    # N.B. Regression is only performed on matrix .X
    if CCregr == "IndPhase":
        print("Regressing individual phases")
        sc.pp.scale(obj)
        sc.tl.score_genes_cell_cycle(obj, s_genes=S_phase_genes, g2m_genes=G2M_phase_genes)
        sc.pp.regress_out(obj, ['S_score', 'G2M_score'])
        sc.pp.scale(obj)
        # Store "old_phase" for later plotting of regression effect on phase%
        obj.obs['OldPhase'] = obj.obs['phase']
        sc.tl.score_genes_cell_cycle(obj, s_genes=S_phase_genes, g2m_genes=G2M_phase_genes)
        print("Regressed")
        print(obj.X[:10,:10])
    elif CCregr == "PhaseDiff":
        print("Regressing phases difference")
        sc.pp.scale(obj)
        sc.tl.score_genes_cell_cycle(obj, s_genes=S_phase_genes, g2m_genes=G2M_phase_genes)
        obj.obs["CC_Difference"] = obj.obs["S_score"] - obj.obs["G2M_score"]
        sc.pp.regress_out(obj, ["CC_Difference"])
        sc.pp.scale(obj)
        obj.obs['OldPhase'] = obj.obs['phase']
        sc.tl.score_genes_cell_cycle(obj, s_genes=S_phase_genes, g2m_genes=G2M_phase_genes)
        print("Regressed")
        print(obj.X[:10,:10])
    elif CCregr == None:
        print("No regression performed")
        sc.pp.scale(obj)
        sc.tl.score_genes_cell_cycle(obj, s_genes=S_phase_genes, g2m_genes=G2M_phase_genes)
    
    return(obj)

def ClusteringOperations(obj, 
                         ExportTables2csv, 
                         plotTopMarkersHeatmap=False, 
                         DisplayTables=False, 
                         csv_suffix=None, 
                         GO_enrichment=True, 
                         PlotCycleTermsProportions=True):
    
    sc.tl.louvain(obj, flavor='vtraag', directed=True,
                  use_weights=True, resolution = ClusteringResolution)
    sc.tl.rank_genes_groups(obj, groupby = 'louvain', use_raw = False)
    if plotTopMarkersHeatmap:
        sc.pl.rank_genes_groups_matrixplot(obj)
    TopMarkersTable = pd.DataFrame(obj.uns['rank_genes_groups']['names'][0:20])
    if DisplayTables:
        print(TopMarkersTable)
    if ExportTables2csv:
        TopMarkersTable.to_csv("TopMarkers_"+csv_suffix+".csv")

    if GO_enrichment:
        df_zero = pd.DataFrame()
        for cluster_number in range(len(
            pd.DataFrame(obj.uns['rank_genes_groups']['names'][0:20]).columns)):
            df = gp.profile(organism='hsapiens',
                            query=list(
                                pd.DataFrame(
                                    obj.uns['rank_genes_groups'][
                                        'names'][0:20]).iloc[
                                    :,cluster_number])).iloc[:10,:].loc[:,["native",
                                                                           "name",
                                                                           "p_value",
                                                                           "term_size",
                                                                           "query_size",
                                                                           "intersection_size"]]
            df['Cluster'] = [cluster_number]*df.shape[0]
            df_zero = pd.concat([df_zero, df])
            
        if PlotCycleTermsProportions:
            CycleTerms = pd.DataFrame(df_zero.name.str.contains(
                "cycle|mitotic|chromosom|nuclear|chromatid|M |phase|division|G1/S", 
                case=False).groupby(df_zero["Cluster"]).sum())
            CycleTerms['Terms'] = ["CycleTerms"]*len(CycleTerms)
            TotTerms = pd.DataFrame(df_zero.name.groupby(df_zero["Cluster"]).count())
            TotTerms['Terms'] = ["TotTerms"]*len(TotTerms)
            df = pd.DataFrame(pd.concat([CycleTerms, TotTerms], axis=0))
            df.reset_index(inplace=True)
            df.columns = ["Cluster","Count", "Terms"]
            df['Cluster'] = df['Cluster'].astype(int)
            df['Count'] = df['Count'].astype(int)
            sns.set(font_scale = 0.9)
            sns.catplot(data=df, x="Cluster", y="Count", hue="Terms", kind="bar")
            
        if DisplayTables:
            print(df_zero)
            
        if ExportTables2csv:
            df_zero.to_csv("GOterms_"+csv_suffix+".csv")
            
    return(obj)
            
def DimensionalityReduction_and_Clustering(obj, 
                                           PCAplots=False, 
                                           DataSetIntegration=None, 
                                           CCregr=None,
                                           ClusteringTopGenesHeatmap=False,
                                           DisplayClusteringTables=False, 
                                           ExportClusteringTables=False, 
                                           analysed_timepoint="FullData",
                                           GOterm=False): 
    # DataSetIntegration is one of  / "MNN" / 
    # CCregr is one of "IndPhase" /
    # analysed_timepoint is one of "FullData" (default) / "d012" / "d14". It needs to be provided if ExportClusteringTables == True
    
    num_PCs = 15

        
    if DataSetIntegration == "MNN":
        
        print("Performing MNN")
        Datasets_list = []
        for ID in np.unique(obj.obs["SampleID"]):
            Datasets_list.append(obj[np.where(obj.obs.loc[:,"SampleID"] == ID)[0],:])
        if analysed_timepoint == "d14": # in case of day 14
            obj = sce.pp.mnn_correct(Datasets_list[0], 
                                     Datasets_list[1], 
                                     Datasets_list[2], batch_key = "SampleID")[0]
        elif analysed_timepoint == "d012": # for day 0, 1, 2
            obj = sce.pp.mnn_correct(Datasets_list[0], 
                                     Datasets_list[1], 
                                     Datasets_list[2], 
                                     Datasets_list[3], 
                                     Datasets_list[4], batch_key = "SampleID")[0]
        elif analysed_timepoint == "FullData": # for day 0, 1, 2
            obj = sce.pp.mnn_correct(Datasets_list[0], 
                                     Datasets_list[4], 
                                     Datasets_list[5], 
                                     Datasets_list[6], 
                                     Datasets_list[7],
                                     Datasets_list[1], 
                                     Datasets_list[2], 
                                     Datasets_list[3], batch_key = "SampleID")[0] #,mnn_order=['d0es', 'd1d', 'd1v', 'd2d', 'd2v', 'd14d-1', 'd14d-2', 'd14d-3']
        obj.obs_names = [x[:-2] for x in obj.obs_names] # remove the suffix MNN put to obs_names as it messess with the exporting later
        
        obj = CellCycleRegression(obj, CCregr)
        print("Performing PCA")
        sc.tl.pca(obj, n_comps=50, zero_center=None, svd_solver='auto', use_highly_variable=None) # hvg only if there
        if PCAplots:
            PCA_plotting_wrapper(obj)
            sc.pl.pca(obj, color = ['OTX2', 'GBX2'], legend_loc = 'on data', use_raw = False)
        sc.pp.scale(obj)
        print("Computing knn")
        sc.pp.neighbors(obj, n_neighbors=30, n_pcs=num_PCs, method='umap', use_rep = 'X_pca')

    obj = ClusteringOperations(obj, 
                               plotTopMarkersHeatmap = ClusteringTopGenesHeatmap,
                               DisplayTables = DisplayClusteringTables, 
                               ExportTables2csv = ExportClusteringTables, 
                               csv_suffix = str(
                                   analysed_timepoint)+"_"+str(DataSetIntegration)+"_"+str(CCregr), 
                               GO_enrichment = GOterm,
                               PlotCycleTermsProportions = GOterm)
    sc.tl.umap(obj, spread=1.0, min_dist=0.5)
    
    return(obj)

def Export_UMAP_clustering(obj, ExportEmbeddingClusteringInfo_suffix):
    
    np.savetxt("ScanpyObsNames_"+ExportEmbeddingClusteringInfo_suffix+".txt",
               np.array(obj.obs_names), delimiter=",", fmt='%s')
    np.savetxt("ScanpyLouvainClustering_"+ExportEmbeddingClusteringInfo_suffix+".txt",
               obj.obs.loc[:,'louvain'].reset_index(), delimiter=",", fmt='%s')
    np.savetxt("ScanpyUMAPCoord_"+ExportEmbeddingClusteringInfo_suffix+".txt",
               obj.obsm['X_umap'], delimiter=",", fmt='%s')

In [ ]:
##########COMBINE INPUT LOOM FILES IF REQUIRED, THEN READ########


if Need_to_combine:
    loompy.combine([FILE_DIR+"/"+name+"-5000_cells.loom" for name in Experiment_names], 
                   OUTPUT_FILE_DIR+"/"+loompy_output,
                   key="Accession")
    
Scanpy_object = sc.read_loom(OUTPUT_FILE_DIR+"/"+loompy_output, sparse=True, cleanup=False, X_name="matrix")
while not Scanpy_object.var_names[Scanpy_object.var_names.duplicated()].empty:
    print("Variables names not unique yet")
    Scanpy_object.var_names_make_unique()
print("Variables names now unique")

In [ ]:
### PRE-PROCESSING
FullDataset = Scanpy_object.copy()

FullDataset = PreProcessing(FullDataset, QCplots=True)

In [ ]:
### FULL DATA: Nothing | BBKNN | MNN | BBKNN Ind | MNN Ind

In [ ]:
def Embedding_clustering_wrapper_Full(Arguments): 
    
    Full_obj = FullDataset.copy()
    Full_obj = DimensionalityReduction_and_Clustering(Full_obj, 
                                                             PCAplots=False, 
                                                             DataSetIntegration=Arguments[0], 
                                                             CCregr=Arguments[1],
                                                             ClusteringTopGenesHeatmap=False,
                                                             DisplayClusteringTables = False, 
                                                             ExportClusteringTables=False, 
                                                             analysed_timepoint="FullData",
                                                             GOterm=True)
    Full_obj.obs["SampleID"] = FullDataset.obs["SampleID"]
    
    return(Full_obj)

Integration_Regression_full = [("MNN", "IndPhase")]
Full_list = [*map(Embedding_clustering_wrapper_Full, Integration_Regression_full)]

PlotTitleList_full = [ "Full data, MNN, IndPhase"]

for i in range(len(Full_list)):
    sc.pl.umap(Full_list[i], color='louvain', legend_loc='on data', use_raw=False)
    sc.pl.umap(Full_list[i], color = ['SampleID', 'OTX2', 'GBX2', 'PAX6', 'FGF17','DCX', 'phase'], 
               title = [PlotTitleList_full[i], 'OTX2', 'GBX2', 'PAX6', 'FGF17','DCX', 'phase'], 
               legend_loc = 'right margin', use_raw = False, ncols=2)

In [ ]:
################SEPARATE THE DATASETS and preprocess#########

In [ ]:
Scanpy_object = Add_obs(Scanpy_object)

d012 = Scanpy_object[np.where(Scanpy_object.obs.loc[:,"SampleID"].isin(["d0es","d1d","d1v","d2d","d2v"]))[0],:]
d14 = Scanpy_object[np.where(Scanpy_object.obs.loc[:,"SampleID"].isin(["d14d-1","d14d-2","d14d-3"]))[0],:]

d012 = PreProcessing(d012, QCplots=True)
d14 = PreProcessing(d14, QCplots=True)

In [ ]:
#################DAY 0,1,2: BBKNN | MNN | BBKNN Ind | MNN Ind | BBKNN Diff | MNN Diff###########
def Embedding_clustering_wrapper_d012(Arguments): 
    
    d012_obj = d012.copy()
    d012_obj = DimensionalityReduction_and_Clustering(d012_obj, 
                                                             PCAplots=False, 
                                                             DataSetIntegration=Arguments[0], 
                                                             CCregr=Arguments[1],
                                                             ClusteringTopGenesHeatmap=False,
                                                             DisplayClusteringTables = False, 
                                                             ExportClusteringTables=True, 
                                                             analysed_timepoint="d012",
                                                             GOterm=True)
    d012_obj.obs["SampleID"] = d012.obs["SampleID"]
    
    return(d012_obj)

#Integration_Regression_d012 = [("BBKNN", None), ("MNN", None), ("BBKNN", "IndPhase"), ("MNN", "IndPhase"), ("BBKNN", "PhaseDiff"), ("MNN", "PhaseDiff")]
Integration_Regression_d012 = [ ("MNN", "IndPhase")]
d012_list = [*map(Embedding_clustering_wrapper_d012, Integration_Regression_d012)]


#PlotTitleList_d012 = ["Day 0,1,2, BBKNN", "Day 0,1,2, MNN", "Day 0,1,2, BBKNN, IndPhase", "Day 0,1,2, MNN, IndPhase", "Day 0,1,2, BBKNN, PhaseDiff", "Day 0,1,2, MNN, PhaseDiff"]
PlotTitleList_d012 = [ "Day 0,1,2, MNN, IndPhase"]
for i in range(len(d012_list)):
    sc.pl.umap(d012_list[i], color='louvain', legend_loc='on data', use_raw=False)
    #sc.pl.umap(d012_list[i], color = ['SampleID', 'OTX2', 'GBX2', 'phase'], 
    sc.pl.umap(d012_list[i], color = [ 'OTX2', 'GBX2'], 
               title = [PlotTitleList_d012[i], 'OTX2', 'GBX2', 'phase'], 
               legend_loc = 'right margin', use_raw = False, ncols=3)

In [ ]:
##############AY 14: BBKNN | MNN | BBKNN Ind | MNN Ind | BBKNN Diff | MNN Diff############

In [ ]:
def Embedding_clustering_wrapper_d14(Arguments): 
    
    d14_obj = d14.copy()
    d14_obj = DimensionalityReduction_and_Clustering(d14_obj, 
                                                             PCAplots=False, 
                                                             DataSetIntegration=Arguments[0], 
                                                             CCregr=Arguments[1],
                                                             ClusteringTopGenesHeatmap=False,
                                                             DisplayClusteringTables = False, 
                                                             ExportClusteringTables=True, 
                                                             analysed_timepoint="d14",
                                                             GOterm=True)
    d14_obj.obs["SampleID"] = d14.obs["SampleID"]
    
    return(d14_obj)

#Integration_Regression_d14 = [("BBKNN", None), ("MNN", None), ("BBKNN", "IndPhase"), ("MNN", "IndPhase"), ("BBKNN", "PhaseDiff"), ("MNN", "PhaseDiff")]
Integration_Regression_d14 = [("MNN", "IndPhase")]
d14_list = [*map(Embedding_clustering_wrapper_d14, Integration_Regression_d14)]

#PlotTitleList_d14 = ["Day 14, BBKNN", "Day 14, MNN", "Day 14, BBKNN, IndPhase", "Day 14, MNN, IndPhase", "Day 14, BBKNN, PhaseDiff", "Day 14, MNN, PhaseDiff"]
PlotTitleList_d14 = [ "Day 14, MNN, IndPhase"]
for i in range(len(d14_list)):
    sc.pl.umap(d14_list[i], color='louvain', legend_loc='on data', use_raw=False)
    sc.pl.umap(d14_list[i], color = ['SampleID', 'OTX2', 'GBX2', 'PAX6', 'FGF17','DCX', 'phase'], 
               title = [PlotTitleList_d14[i], 'OTX2', 'GBX2', 'PAX6', 'FGF17','DCX', 'phase'], 
               legend_loc = 'right margin', use_raw = False, ncols=2)

In [ ]:
######EXPORT UMAP AND CLUSTERING INFORMATION FOR EXTERNAL ANALYSIS

In [ ]:
suffix_list = [ "d012_MNN_IndRegr"]
for n in range(len(suffix_list)):
    Export_UMAP_clustering(d012_list[i], suffix_list[n])
    
#suffix_list = [ "d14_MNN_IndRegr"]
#for n in range(len(suffix_list)):
#    Export_UMAP_clustering(d14_list[i], suffix_list[n])

In [ ]:
####################FROM HERE ON IS VELOCITY##############

In [ ]:
import scvelo as scv
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.set_figure_params('scvelo')
scv.logging.print_version()

In [ ]:
def Dataset_loading_and_cleanup(dataset):
    # dataset is one of "FullDataset" | "d012" | "d14"
    
    FullDataset = Scanpy_object.copy()
    FullDataset = Add_obs(FullDataset)
    
    if dataset == "d012":
        adata = FullDataset[np.where(FullDataset.obs.loc[:,"SampleID"].isin(["d0es","d1d","d1v","d2d","d2v"]))[0],:]
    elif dataset == "d14":
        adata = FullDataset[np.where(FullDataset.obs.loc[:,"SampleID"].isin(["d14d-1","d14d-2","d14d-3"]))[0],:]
    elif dataset == "FullDataset":
        adata = FullDataset.copy()
    del(FullDataset)
    
    print("Cleaning "+str(dataset))
    scv.utils.cleanup(adata)
    #scv.utils.clean_obs_names(adata)
    while not adata.var_names[adata.var_names.duplicated()].empty:
        print("Variables names not unique yet")
        adata.var_names_make_unique()
    print("Variables names now unique")
    print(adata)
    scv.utils.show_proportions(adata)
    
    return(adata)

def Velocity_PreProcessing_UMAP_Clustering(obj, Scanpy_obj):
    # UMAP and Clustering information is passed from one of the objects processed above,
    # to be indicated with the Scanpy_obj argument
    
    scv.pp.filter_and_normalize(obj, min_shared_counts=10, n_top_genes=3000)
    median_genes_per_cell = np.median(obj.X.getnnz(axis=1)) # recomputing this per each dataset will introduce inconsistent thrholds
    sc.pp.filter_cells(obj, max_genes=2.5*median_genes_per_cell)
    obj.obs['sample_batch'] = [name[0] for name in obj.obs_names.str.split("-5")]
    print("After filtering: "+str(obj.shape))
    scv.pp.moments(obj, n_neighbors=30, n_pcs=15) # n_pcs=30 by default
    
    # In order to transfer information from Scanpy and adata, I will subset for the intersection of the two in terms of cells
    # I will only transfer louvain and X_umap coordinates
    obj = obj[obj.obs_names.intersection(Scanpy_obj.obs_names).to_list(),:]
    MatchingIndexes = [Scanpy_obj.obs_names.to_list().index(x) for x in obj.obs_names.intersection(Scanpy_obj.obs_names).to_list()]
    SampleIDs = Scanpy_obj.obs.iloc[MatchingIndexes,:].loc[:,'SampleID']
    LouvainClusters = Scanpy_obj.obs.iloc[MatchingIndexes,:].loc[:,'louvain']
    UmapCoordinates = Scanpy_obj.obsm['X_umap'][MatchingIndexes,:]
    obj.obs['SampleID'] = ["d2d" if ID == "48hrsd" else 
                           "d14d" if ID == "d14d-1" else
                           "d14d" if ID == "d14d-2" else
                           "d14d" if ID == "d14d-3" else
                           ID for ID in SampleIDs]
    obj.obs['louvain'] = LouvainClusters
    obj.obsm['X_umap'] = UmapCoordinates
    
    return(obj)

def Velocity_Computations(obj, plotTitle, plotGenes=False):
    scv.tl.velocity(obj, mode='stochastic', use_raw = False) # , groupby='SampleID' --> if I use this  {ValueError: You need to run `pp.neighbors` first to compute a neighborhood graph.}
    if plotGenes:
        scv.pl.velocity(obj, var_names=['OTX2', 'GBX2', 'CRABP2', 'DDIT4','SOX10', 'PAX6','DCX', 'FGF17'])
    scv.tl.velocity_graph(obj)
    scv.tl.velocity_embedding(obj, basis='umap')
    scv.pl.velocity_embedding_stream(obj, color=['SampleID', 'louvain'], title = [plotTitle, plotTitle], basis='umap', vkey='velocity', density=1, legend_loc='on data')
    
    return(obj)

In [ ]:
#################DAY 012 VELOCITIES#####
d012_Velocity = Dataset_loading_and_cleanup("d012")

def VelocityWrapper_d012(args):
    ScanpyObject = args[0]
    plot_title = args[1]
    scVelo_obj = d012_Velocity.copy()
    scVelo_obj = Velocity_PreProcessing_UMAP_Clustering(scVelo_obj, Scanpy_obj=ScanpyObject)
    scVelo_obj = Velocity_Computations(scVelo_obj, plotTitle = plot_title)
    return(scVelo_obj)

VelocityArguments_d012 = [
                     (d012_list[i], "Day 0,1,2, MNN, IndPhase"),
                     
]
VelocityResults_d012 = [*map(VelocityWrapper_d012, VelocityArguments_d012)]

In [ ]:
def Velocity_Computations(obj, plotTitle, plotGenes=False):
    obj.obs["custom_color"] = ["darkgrey" for i in range(obj.obs.shape[0])]
    scv.tl.velocity(obj, mode='stochastic', use_raw = False) # , groupby='SampleID' --> if I use this  {ValueError: You need to run `pp.neighbors` first to compute a neighborhood graph.}
    if plotGenes:
        scv.pl.velocity(obj, var_names=['OTX2', 'GBX2', 'CRABP2', 'DDIT4','SOX10', 'PAX6','DCX', 'FGF17'])
        sc.pl.umap(obj, color = ['SampleID', 'OTX2', 'GBX2', 'PAX6', 'FGF17','DCX', 'phase'], 
               title = [PlotTitleList_d14[i], 'OTX2', 'GBX2', 'PAX6', 'FGF17','DCX', 'phase'], 
               legend_loc = 'right margin', use_raw = False, ncols=2, palette='BrBG')
    scv.tl.velocity_graph(obj)
    scv.tl.velocity_embedding(obj, basis='umap')
    #scv.pl.velocity_embedding_stream(obj, color=['SampleID', 'louvain'], title = [plotTitle, plotTitle], basis='umap', vkey='velocity', density=1, legend_loc='right margin',save="d012velocity.png")
    scv.pl.velocity_embedding_stream(obj, color="custom_color", title = [plotTitle, plotTitle], basis='umap', vkey='velocity', density=1, legend_loc='right margin',save="d012velocity.png")
    #scv.pl.velocity_embedding(obj, color=['SampleID', 'louvain'], title = [plotTitle, plotTitle],basis='umap', arrow_length=2, arrow_size=1.5)
    scv.pl.velocity_embedding(obj, title = [plotTitle, plotTitle],basis='umap', arrow_length=2, arrow_size=1.5)
    sc.pl.umap(obj, color = ['SampleID', 'OTX2', 'GBX2', 'PAX6', 'FGF17','DCX', 'phase'], 
               legend_loc = 'right margin', use_raw = False, ncols=2, palette='BrBG')
    return(obj)

In [ ]:
###############DAY 14 VELOCITIES############
d14_Velocity = Dataset_loading_and_cleanup("d14")

def VelocityWrapper_d14(args):
    ScanpyObject = args[0]
    plot_title = args[1]
    scVelo_obj = d14_Velocity.copy()
    scVelo_obj = Velocity_PreProcessing_UMAP_Clustering(scVelo_obj, Scanpy_obj=ScanpyObject)
    scVelo_obj = Velocity_Computations(scVelo_obj, plotTitle = plot_title)
    return(scVelo_obj)

VelocityArguments_d14 = [#(d14_list[0], "Day 14, BBKNN"),
                     #(d14_list[1], "Day 14, MNN"),
                     #(d14_list[2], "Day 14, BBKNN, IndPhase"),
                     (d14_list[i], "Day 14, MNN, IndPhase"),
                     #(d14_list[4], "Day 14, BBKNN, PhaseDiff"),
                     #(d14_list[5], "Day 14, MNN, PhaseDiff")]
]
VelocityResults_d14 = [*map(VelocityWrapper_d14, VelocityArguments_d14)]

In [ ]:
FullDataset

In [ ]:
d012